
# Qwen2‑VL Demo + Exercises (Starter Notebook)

This notebook includes:
- **Demo**: Load Qwen2‑VL, (optional) prune, LoRA fine‑tune on CIFAR‑10 as VQA, evaluate, simple inference.
- **Exercise A**: Constrained decoding via candidate scoring (log‑prob aggregation).
- **Exercise B**: Pruning ablation (FFN vs Attn heads) measuring perplexity and images/sec.
- **Exercise C**: Prompt/EOS ablation—measure repetition and accuracy effects.

All student work is in clearly marked **TODO** functions.


## 0) Setup (Installs & GPU)

In [ ]:

# If torch/torchvision already installed, you can skip this pip line.
!pip -q install "transformers>=4.43.3" "accelerate>=0.32.0" "peft>=0.11.1" "datasets" "pillow" "torchvision"


In [ ]:

import os, gc, math, random, platform, time, warnings
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from typing import List

os.environ["TOKENIZERS_PARALLELISM"] = "false"

print("Torch:", torch.__version__, "| CUDA:", torch.version.cuda, "| Py:", platform.python_version())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")

SEED = 42
random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)


## 1) Imports & Config

In [ ]:

from PIL import Image
import torchvision
import torchvision.transforms as T

from transformers import (
    AutoProcessor, AutoConfig, AutoModelForVision2Seq, get_linear_schedule_with_warmup
)
from peft import LoraConfig, get_peft_model

# --- Demo knobs ---
MODEL_NAME   = "Qwen/Qwen2-VL-2B-Instruct"
PRUNE_MODE   = "ffn_channels"               # "attn_heads" or "ffn_channels"
PRUNE_RATIO  = 0.15
USE_FP32     = False                        # FP16 recommended on T4

# LoRA / Train knobs (tiny)
LR                = 1e-4
EPOCHS            = 1
BATCH_SIZE        = 2
GRAD_ACCUM_STEPS  = 4
MAX_LENGTH        = 512
WARMUP_STEPS      = 20
N_TRAIN, N_VAL    = 400, 100  # smaller for faster classroom runs

OUTPUT_DIR        = "./demo_pruned_lora_qwen2vl"
os.makedirs(OUTPUT_DIR, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype  = torch.float32 if USE_FP32 else torch.float16
print(f"Device: {device} | Dtype: {dtype} | Prune: {PRUNE_MODE} @ {int(PRUNE_RATIO*100)}%")

processor = AutoProcessor.from_pretrained(MODEL_NAME, trust_remote_code=True)
config    = AutoConfig.from_pretrained(MODEL_NAME, trust_remote_code=True)

def load_model(dtype):
    try:
        m = AutoModelForVision2Seq.from_pretrained(
            MODEL_NAME, torch_dtype=dtype, low_cpu_mem_usage=True, trust_remote_code=True
        ).to(device)
        return m, dtype
    except torch.cuda.OutOfMemoryError:
        print("OOM at requested dtype; falling back to FP16.")
        torch.cuda.empty_cache(); gc.collect()
        m = AutoModelForVision2Seq.from_pretrained(
            MODEL_NAME, torch_dtype=torch.float16, low_cpu_mem_usage=True, trust_remote_code=True
        ).to(device)
        return m, torch.float16

model, dtype = load_model(dtype)
model.gradient_checkpointing_enable()

hidden_size        = getattr(config, "hidden_size", getattr(config, "hidden_dim", None))
num_heads          = getattr(config, "num_attention_heads", getattr(config, "num_heads", None))
intermediate_size  = getattr(config, "intermediate_size", getattr(config, "ffn_hidden_size", None))
assert hidden_size and num_heads and intermediate_size, "Missing key config dims."
head_dim = hidden_size // num_heads

total_params_m = sum(p.numel() for p in model.parameters())/1e6
print(f"Total params: {total_params_m:.1f}M")


## 2) Pruning Utilities

In [ ]:

def find_attn_modules(module):
    for name, m in module.named_modules():
        if all(hasattr(m, x) for x in ["q_proj","k_proj","v_proj","o_proj"]):
            yield name, m

def find_mlp_modules(module):
    for name, m in module.named_modules():
        if all(hasattr(m, x) for x in ["gate_proj","up_proj","down_proj"]):
            yield name, m

@torch.no_grad()
def prune_attention_heads_logical_gqa(attn_mod, ratio: float):
    n_q  = getattr(attn_mod, "num_heads", None)         or getattr(attn_mod, "n_heads", None)         or getattr(getattr(attn_mod, "config", None), "num_attention_heads", None)
    n_kv = getattr(attn_mod, "num_key_value_heads", None)         or getattr(attn_mod, "n_kv_heads", None)         or getattr(getattr(attn_mod, "config", None), "num_key_value_heads", None)         or n_q
    hd = getattr(attn_mod, "head_dim", None)
    if hd is None:
        q_rows = attn_mod.q_proj.weight.shape[0]
        assert n_q and q_rows % n_q == 0, "Cannot infer head_dim."
        hd = q_rows // n_q
    assert n_q and n_kv and n_q >= 1 and n_kv >= 1
    assert n_q % n_kv == 0, "Expected n_q divisible by n_kv for GQA."

    n_keep_q = max(1, int(n_q * (1.0 - ratio)))
    prune_q  = list(range(n_keep_q, n_q))
    if not prune_q:
        return

    def rows_for_heads(head_ids, per_head):
        rows = []
        for h in head_ids:
            s = h * per_head
            rows.extend(range(s, s + per_head))
        return rows

    rows_q  = rows_for_heads(prune_q, hd)
    group   = max(1, n_q // n_kv)
    prune_kv = sorted(set(h // group for h in prune_q))
    rows_kv = rows_for_heads(prune_kv, hd)

    # q_proj rows -> mask multiply
    Wq = attn_mod.q_proj.weight
    mask_q = torch.ones(Wq.shape[0], device=Wq.device, dtype=Wq.dtype)
    if rows_q:
        idx_q = torch.tensor([r for r in rows_q if 0 <= r < Wq.shape[0]], device=Wq.device, dtype=torch.long)
        if idx_q.numel() > 0:
            mask_q.index_fill_(0, idx_q, 0)
            Wq.mul_(mask_q[:, None])
            bq = getattr(attn_mod.q_proj, "bias", None)
            if bq is not None: bq.mul_(mask_q.to(bq.dtype))

    # k_proj / v_proj rows
    for pname, rows in (("k_proj", rows_kv), ("v_proj", rows_kv)):
        proj = getattr(attn_mod, pname)
        W    = proj.weight
        mask = torch.ones(W.shape[0], device=W.device, dtype=W.dtype)
        if rows:
            idx  = torch.tensor([r for r in rows if 0 <= r < W.shape[0]], device=W.device, dtype=torch.long)
            if idx.numel() > 0:
                mask.index_fill_(0, idx, 0)
                W.mul_(mask[:, None])
                b = getattr(proj, "bias", None)
                if b is not None: b.mul_(mask.to(b.dtype))

    # o_proj columns corresponding to pruned Q rows
    Wo = attn_mod.o_proj.weight
    col_mask = torch.ones(Wo.shape[1], device=Wo.device, dtype=Wo.dtype)
    if rows_q:
        idx_cols = torch.tensor([c for c in rows_q if 0 <= c < Wo.shape[1]], device=Wo.device, dtype=torch.long)
        if idx_cols.numel() > 0:
            col_mask.index_fill_(0, idx_cols, 0)
            Wo.mul_(col_mask[None, :])

@torch.no_grad()
def prune_ffn_channels_logical_mask(mlp_mod: nn.Module, ratio: float, intermediate_size: int):
    n_prune = max(1, int(intermediate_size * ratio))
    if n_prune <= 0: return
    down_W = mlp_mod.down_proj.weight  # [hidden, inter]
    col_norms = torch.norm(down_W, p=1, dim=0)
    prune_idx = torch.topk(col_norms, k=n_prune, largest=False).indices

    inter_dim = down_W.shape[1]
    keep_mask = torch.ones(inter_dim, device=down_W.device, dtype=down_W.dtype)
    if prune_idx.numel() > 0:
        keep_mask.index_fill_(0, prune_idx, 0)

    # down_proj: zero columns
    mlp_mod.down_proj.weight.mul_(keep_mask[None, :])
    # up_proj / gate_proj: zero rows
    for name in ["up_proj","gate_proj"]:
        getattr(mlp_mod, name).weight.mul_(keep_mask[:, None])
        b = getattr(getattr(mlp_mod, name), "bias", None)
        if b is not None:
            b.mul_(keep_mask.to(b.dtype))


## 3) CIFAR‑10 → VQA data (EOS‑supervised answers)

In [ ]:

# tokenizer setup
if processor.tokenizer.pad_token_id is None:
    processor.tokenizer.pad_token = processor.tokenizer.eos_token
PAD_ID   = processor.tokenizer.pad_token_id
IGNOREID = -100
EOS      = processor.tokenizer.eos_token

root = "./data_cifar10"
train_raw = torchvision.datasets.CIFAR10(root=root, train=True, download=True)
test_raw  = torchvision.datasets.CIFAR10(root=root, train=False, download=True)
label_names = train_raw.classes

resize_to = 448
resize_tf = T.Resize((resize_to, resize_to))

def make_examples(ds, n_take):
    items = []
    for i in range(n_take):
        img_pil, y = ds[i]
        img = resize_tf(img_pil)
        ans = label_names[y]
        items.append({
            "image": img,
            "question": "What is in this image?",
            "answer": ans,
            "answer_with_eos": ans + EOS,
        })
    return items

train_items = make_examples(train_raw, N_TRAIN)
val_items   = make_examples(test_raw,  N_VAL)

def encode_example_vqa(ex):
    messages_train = [
        {"role":"user","content":[
            {"type":"image","image": ex["image"]},
            {"type":"text","text": ex["question"]}
        ]},
        {"role":"assistant","content":[{"type":"text","text": ex["answer_with_eos"]}]}
    ]
    messages_gen = [
        {"role":"user","content":[
            {"type":"image","image": ex["image"]},
            {"type":"text","text": ex["question"]}
        ]}
    ]
    train_text = processor.apply_chat_template(messages_train, tokenize=False, add_generation_prompt=False)
    gen_text   = processor.apply_chat_template(messages_gen,   tokenize=False, add_generation_prompt=True)

    out = processor(
        text=[train_text],
        images=[ex["image"]],
        return_tensors="pt",
        max_length=MAX_LENGTH,
        padding="longest",
        truncation=True
    )

    def mask_answer_only(input_ids_2d, answer_text):
        full_ids = input_ids_2d[0].tolist()
        ans_ids  = processor.tokenizer(answer_text, add_special_tokens=False, return_tensors="pt")["input_ids"][0].tolist()
        def find_subseq(a, b):
            L, M = len(a), len(b)
            if M == 0 or M > L: return -1
            for i in range(L - M + 1):
                if a[i:i+M] == b: return i
            return -1
        labels = input_ids_2d.clone(); labels[:] = -100
        start = find_subseq(full_ids, ans_ids)
        if start >= 0:
            end = start + len(ans_ids)
            labels[:, start:end] = input_ids_2d[:, start:end]
        else:
            keep = min(8, input_ids_2d.shape[1])
            labels[:, -keep:] = input_ids_2d[:, -keep:]
        return labels

    input_ids = out["input_ids"]
    labels    = mask_answer_only(input_ids, ex["answer_with_eos"])

    return {
        "pixel_values": out["pixel_values"].squeeze(0),
        "image_grid_thw": out.get("image_grid_thw", None).squeeze(0) if out.get("image_grid_thw", None) is not None else None,
        "input_ids": input_ids.squeeze(0),
        "attention_mask": out["attention_mask"].squeeze(0),
        "labels": labels.squeeze(0),
        "answer_text": ex["answer"],
        "raw_image":   ex["image"],
        "gen_prompt":  gen_text,
    }

train_encoded = [encode_example_vqa(ex) for ex in train_items]
val_encoded   = [encode_example_vqa(ex) for ex in val_items]

class CIFARVQADataset(Dataset):
    def __init__(self, encoded): self.encoded = encoded
    def __len__(self): return len(self.encoded)
    def __getitem__(self, i): return self.encoded[i]

train_ds = CIFARVQADataset(train_encoded)
val_ds   = CIFARVQADataset(val_encoded)

def pad_1d(seqs, pad_val):
    maxlen = max(x.size(0) for x in seqs)
    out = torch.full((len(seqs), maxlen), pad_val, dtype=seqs[0].dtype)
    for i, s in enumerate(seqs):
        out[i, :s.size(0)] = s
    return out

def collate_fn(batch):
    out = {}
    out["pixel_values"]   = torch.stack([b["pixel_values"] for b in batch], dim=0)
    grids = [b["image_grid_thw"] for b in batch]
    out["image_grid_thw"] = torch.stack(grids, dim=0) if all(g is not None for g in grids) else None

    out["input_ids"]      = pad_1d([b["input_ids"] for b in batch], PAD_ID)
    out["attention_mask"] = pad_1d([b["attention_mask"] for b in batch], 0)
    out["labels"]         = pad_1d([b["labels"] for b in batch], IGNOREID)

    out["answer_text"] = [b["answer_text"] for b in batch]
    out["raw_images"]  = [b["raw_image"]   for b in batch]
    out["gen_prompts"] = [b["gen_prompt"]  for b in batch]
    return out

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  collate_fn=collate_fn)
val_loader   = DataLoader(val_ds,   batch_size=2,          shuffle=False, collate_fn=collate_fn)


## 4) Baseline Eval Helpers

In [ ]:

@torch.no_grad()
def eval_loss(model, loader):
    model.eval()
    total_loss, total_tokens = 0.0, 0
    for batch in loader:
        moved = {}
        for k, v in batch.items():
            if k == "pixel_values" and v is not None:
                moved[k] = v.to(device, dtype=dtype)
            elif k in ("input_ids","attention_mask","labels") and v is not None:
                moved[k] = v.to(device)
        if batch.get("image_grid_thw") is not None:
            moved["image_grid_thw"] = batch["image_grid_thw"].to(device)
        out = model(**moved)
        n_tokens = (moved["labels"] != -100).sum().item()
        total_loss += out.loss.item() * max(1, n_tokens)
        total_tokens += max(1, n_tokens)
    model.train()
    import math
    return math.exp(total_loss / max(1, total_tokens))

@torch.no_grad()
def eval_gen_accuracy(model, processor, loader, k_samples=50, max_new_tokens=3):
    model.eval()
    correct, seen = 0, 0
    for batch in loader:
        for img, gp, gold in zip(batch["raw_images"], batch["gen_prompts"], batch["answer_text"]):
            if seen >= k_samples: break
            enc = processor(text=[gp], images=[img], return_tensors="pt")
            enc = {k: v.to(device) for k, v in enc.items()}
            gen_ids = model.generate(
                **enc,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                num_beams=1,
                pad_token_id=processor.tokenizer.eos_token_id,
                eos_token_id=processor.tokenizer.eos_token_id,
                repetition_penalty=1.5,
                length_penalty=2.0,
                use_cache=True,
            )
            new_tokens = gen_ids[:, enc["input_ids"].shape[1]:]
            text = processor.batch_decode(new_tokens, skip_special_tokens=True)[0].strip().lower()
            pred = text.split()[0] if text else ""
            if gold.lower() in pred:
                correct += 1
            seen += 1
        if seen >= k_samples: break
    model.train()
    return correct / max(1, seen)

print("Quick baseline eval (pre‑prune)…")
ppl_train = eval_loss(model, train_loader)
acc_val0  = eval_gen_accuracy(model, processor, val_loader, k_samples=30)
print(f"Train PPL (pre‑prune): {ppl_train:.2f} | Val Gen@1 Acc (pre‑prune): {acc_val0:.2%}")


## 5) Apply Pruning

In [ ]:

if PRUNE_MODE == "attn_heads":
    n = 0
    for _, attn in find_attn_modules(model):
        prune_attention_heads_logical_gqa(attn, PRUNE_RATIO)
        n += 1
    print(f"Pruned heads in {n} attention modules (mask‑based).")
elif PRUNE_MODE == "ffn_channels":
    n = 0
    for _, mlp in find_mlp_modules(model):
        prune_ffn_channels_logical_mask(mlp, PRUNE_RATIO, intermediate_size)
        n += 1
    print(f"Pruned channels in {n} MLP modules (mask‑based).")
else:
    raise ValueError("PRUNE_MODE must be 'attn_heads' or 'ffn_channels'.")

ppl_after_prune = eval_loss(model, train_loader)
print(f"PPL (post‑prune, pre‑LoRA): {ppl_after_prune:.2f}")


## 6) LoRA Setup & Tiny Fine‑Tune

In [ ]:

def collect_lora_targets(m: nn.Module) -> List[str]:
    names = set()
    for n, mod in m.named_modules():
        if isinstance(mod, nn.Linear):
            for key in ["q_proj","k_proj","v_proj","o_proj","up_proj","down_proj","gate_proj"]:
                if n.endswith(key): names.add(n.split(".")[-1])
    return sorted(list(names)) or ["q_proj","k_proj","v_proj","o_proj","up_proj","down_proj","gate_proj"]

targets = collect_lora_targets(model)
print("LoRA targets:", targets)

lora_cfg = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05,
    bias="none", task_type="CAUSAL_LM",
    target_modules=targets
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()

optimizer = torch.optim.AdamW((p for p in model.parameters() if p.requires_grad), lr=LR, weight_decay=0.0)
steps_per_epoch  = math.ceil(len(train_loader) / GRAD_ACCUM_STEPS)
num_training_steps = EPOCHS * steps_per_epoch
sched = get_linear_schedule_with_warmup(optimizer, WARMUP_STEPS, num_training_steps)

scaler = torch.cuda.amp.GradScaler(enabled=(dtype==torch.float16))
model.train()
global_step = 0

for epoch in range(EPOCHS):
    for step, batch in enumerate(train_loader):
        moved = {}
        for k, v in batch.items():
            if k == "pixel_values":
                moved[k] = v.to(device, dtype=dtype)
            elif k in ("input_ids","attention_mask","labels"):
                moved[k] = v.to(device)
        if batch.get("image_grid_thw") is not None:
            moved["image_grid_thw"] = batch["image_grid_thw"].to(device)

        with torch.cuda.amp.autocast(enabled=(dtype==torch.float16)):
            out = model(**moved)
            loss = out.loss / GRAD_ACCUM_STEPS

        scaler.scale(loss).backward()

        if (step + 1) % GRAD_ACCUM_STEPS == 0:
            scaler.step(optimizer); scaler.update()
            optimizer.zero_grad(set_to_none=True)
            sched.step()
            global_step += 1
            if global_step % 5 == 0:
                print(f"step {global_step} | loss={(loss.item()*GRAD_ACCUM_STEPS):.4f}")

torch.cuda.empty_cache()


In [ ]:

ppl_after_lora = eval_loss(model, train_loader)
acc_val1       = eval_gen_accuracy(model, processor, val_loader, k_samples=30)
print(f"PPL (post‑LoRA): {ppl_after_lora:.2f} | Val Gen@1 Acc (post‑LoRA): {acc_val1:.2%}")

# Save LoRA + merged
model.save_pretrained(os.path.join(OUTPUT_DIR, "lora_adapter"))
processor.save_pretrained(OUTPUT_DIR)
print("Saved LoRA adapter to:", os.path.join(OUTPUT_DIR, "lora_adapter"))

print("Merging LoRA into base (uses extra VRAM)…")
merged = model.merge_and_unload()
merged.save_pretrained(os.path.join(OUTPUT_DIR, "merged_full"))
print("Merged checkpoint saved to:", os.path.join(OUTPUT_DIR, "merged_full"))


## 7) Inference Demo (Greedy)

In [ ]:

def qwen_vl_infer(model, processor, pil_image, question: str, max_new_tokens=3):
    pil_image = pil_image.resize((448, 448))
    messages = [{"role":"user","content":[
        {"type":"image","image": pil_image},
        {"type":"text","text": question}
    ]}]
    prompt = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = processor(text=[prompt], images=[pil_image], return_tensors="pt")
    for k, v in list(inputs.items()):
        if k == "pixel_values":
            inputs[k] = v.to(device, dtype=dtype)
        else:
            inputs[k] = v.to(device)

    gen_ids = model.generate(
        **inputs,
        max_new_tokens=3,
        do_sample=False,
        num_beams=1,
        pad_token_id=processor.tokenizer.eos_token_id,
        eos_token_id=processor.tokenizer.eos_token_id,
        repetition_penalty=1.5,
        length_penalty=2.0,
        use_cache=True
    )
    new_tokens = gen_ids[:, inputs["input_ids"].shape[1]:]
    text = processor.batch_decode(new_tokens, skip_special_tokens=True)[0].strip()
    return text

print("\n--- Demo predictions (merged model) ---")
for i in range(5):
    img_pil, y = test_raw[i]
    ans = label_names[y]
    pred = qwen_vl_infer(merged, processor, img_pil, "What is in this image?")
    print(f"GT: {ans:<10s} | PRED: {pred}")


## Exercise A — Constrained Decoding (Candidate Scoring)


**Goal:** Always answer with a CIFAR‑10 class name by scoring each candidate and choosing the argmax.

Implement:
- `score_answer_logprob(model, processor, pil_img, question, answer)`
- `predict_with_candidates(model, processor, pil_img, question, candidates)`

Hints:
1. Build the user‑only prompt as in the demo (with `add_generation_prompt=True`).
2. Tokenize candidate `answer` with the tokenizer only (no image), `add_special_tokens=False`.
3. Accumulate log‑probs by feeding the next gold token and summing `log_softmax(logits[:, -1])`.


In [ ]:

from torch.nn.functional import log_softmax

CANDIDATES = [c.lower() for c in label_names]

def score_answer_logprob(model, processor, pil_img, question, answer) -> float:
    """TODO A1: Return total conditional log‑prob of 'answer' tokens given (image, question).
    Do teacher‑forcing over the answer tokens; do NOT free‑generate.
    """
    # 1) Build base prompt from (image, question)
    # 2) Tokenize base once (inputs)
    # 3) Tokenize 'answer' with tokenizer only, add_special_tokens=False
    # 4) Iteratively:
    #    - forward with current context
    #    - take logits[:, -1], compute log_softmax, add logp of next token
    #    - append the next token to the input_ids (and update attention_mask)
    # 5) Return the float total
    pass

def predict_with_candidates(model, processor, pil_img, question, candidates) -> str:
    """TODO A2: Use score_answer_logprob over candidates and return best string."""
    pass

# Quick smoke (will fail until implemented)
try:
    _ = predict_with_candidates(merged, processor, test_raw[0][0], "What is in this image?", CANDIDATES)
    print("Exercise A smoke test ran.")
except Exception as e:
    print("Exercise A: implement TODOs first ->", e)


## Exercise B — Pruning Ablation (Quality & Speed)


**Goal:** Compare `ffn_channels` vs `attn_heads` at ratios (0.15, 0.30). Report:
- Train PPL (no LoRA fine‑tune for speed)
- Images/sec using your candidate predictor from Exercise A on ~64 samples

Implement:
- `apply_pruning(model, mode, ratio)`
- `measure_decode_ips(model, processor, val_loader, n_samples=64)`

Use a **fresh** base model for each setting via `load_model(dtype)`.


In [ ]:

SETTINGS = [
    ("ffn_channels", 0.15),
    ("ffn_channels", 0.30),
    ("attn_heads",   0.15),
    ("attn_heads",   0.30),
]

def apply_pruning(model, mode: str, ratio: float):
    """TODO B1: Apply FFN‑channel or attention‑head pruning in‑place (use helpers above)."""
    pass

@torch.no_grad()
def measure_decode_ips(model, processor, val_loader, n_samples=64) -> float:
    """TODO B2: Time candidate predictions over n_samples and return images/sec (float)."""
    pass

ablation_results = []
for mode, ratio in SETTINGS:
    base_model, _ = load_model(dtype)  # fresh base
    base_model.eval()
    apply_pruning(base_model, mode, ratio)
    ppl = eval_loss(base_model, train_loader)
    try:
        ips = measure_decode_ips(base_model, processor, val_loader, n_samples=64)
    except Exception as e:
        ips = float("nan")
        print(f"[{mode}@{ratio}] measure_decode_ips error:", e)
    ablation_results.append({"mode": mode, "ratio": ratio, "ppl": ppl, "img_per_sec": ips})
    print(ablation_results[-1])

ablation_results


## Exercise C — Prompt / EOS Ablation (Repetition vs Accuracy)


**Goal:** Show why adding EOS to the supervised answer helps reduce repetition.

Implement:
- `make_examples_with_eos_flag(ds, n_take, use_eos)`
- `compute_repetition_rate(texts)`

Suggested quick protocol:
1) Build two small validation sets (e.g., 60 samples) with `use_eos=True/False`.
2) Evaluate greedy decoding on both with your merged model.
3) Compare accuracy and repetition rate.


In [ ]:

def make_examples_with_eos_flag(ds, n_take, use_eos=True):
    """TODO C1: Return list of dicts with keys: image, question, answer, answer_with_eos (if use_eos)."""
    pass

def compute_repetition_rate(texts):
    """TODO C2: Return fraction of outputs exhibiting simple repetition (e.g., duplicated first token/bigram)."""
    pass

SMALL_N = 60
try:
    val_eos   = make_examples_with_eos_flag(test_raw, SMALL_N, use_eos=True)
    val_noeos = make_examples_with_eos_flag(test_raw, SMALL_N, use_eos=False)
except Exception as e:
    print("Exercise C (dataset build) -> implement TODOs first:", e)
    val_eos, val_noeos = [], []

def quick_encode_list(items):
    out = []
    for ex in items:
        ex2 = dict(ex)
        if ex2.get("answer_with_eos") is None:
            ex2["answer_with_eos"] = ex2["answer"]
        out.append(encode_example_vqa(ex2))
    return out

if val_eos and val_noeos:
    val_eos_enc   = quick_encode_list(val_eos)
    val_noeos_enc = quick_encode_list(val_noeos)

    def decode_set(merged_model, items, k=60):
        preds, golds = [], []
        for i, ex in enumerate(items[:k]):
            img = ex["image"]
            ans = ex["answer"]
            msg = [{"role":"user","content":[{"type":"image","image": img},{"type":"text","text": "What is in this image?"}]}]
            gp  = processor.apply_chat_template(msg, tokenize=False, add_generation_prompt=True)
            enc = processor(text=[gp], images=[img], return_tensors="pt")
            enc = {k: v.to(device) for k, v in enc.items()}
            gen_ids = merged_model.generate(
                **enc, max_new_tokens=3, do_sample=False, num_beams=1,
                pad_token_id=processor.tokenizer.eos_token_id,
                eos_token_id=processor.tokenizer.eos_token_id,
            )
            new_tokens = gen_ids[:, enc["input_ids"].shape[1]:]
            text = processor.batch_decode(new_tokens, skip_special_tokens=True)[0].strip().lower()
            preds.append(text)
            golds.append(ans.lower())
        return preds, golds

    try:
        preds_eos,   golds_eos   = decode_set(merged, val_eos)
        preds_noeos, golds_noeos = decode_set(merged, val_noeos)
        acc_eos   = sum(int(p.split()[0] in g) for p,g in zip(preds_eos, golds_eos)) / max(1,len(golds_eos))
        acc_noeos = sum(int(p.split()[0] in g) for p,g in zip(preds_noeos, golds_noeos)) / max(1,len(golds_noeos))
        rep_eos   = compute_repetition_rate(preds_eos)
        rep_noeos = compute_repetition_rate(preds_noeos)
        print({"acc_eos":acc_eos, "acc_noeos":acc_noeos, "rep_eos":rep_eos, "rep_noeos":rep_noeos})
    except Exception as e:
        print("Exercise C: implement TODOs first ->", e)
else:
    print("Exercise C waiting for TODO implementations.")
